In [ ]:
import imageio
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
data = pd.read_csv("hannah_data.csv")
data["density"] = data["tot_length"]/data["area_sep_comp"]

In [ ]:
im = imageio.imread("hilbert.PNG")

In [ ]:
plt.imshow(im[:,:,0]>100)

In [ ]:
thersholded = im[:,:,0]>100
from skimage.feature import corner_fast
corners = corner_fast(im[:,:,0],threshold=0.05,)
points = corners.nonzero()


In [ ]:
np.array((points[1],points[0])).transpose()

In [ ]:
plt.imshow(im)
plt.scatter(points[1],points[0])

In [ ]:
from skimage.morphology import skeletonize
from skimage import measure

In [ ]:
def is_corner(points):
    """
    Returns True if the angle at the middle point of the input points
    is approximately 90 degrees, False otherwise.
    """
    middle_point = points[len(points)//2]
    angle_tol = np.deg2rad(20)
    angle_diff = np.abs(np.diff(np.arctan2(points[:,0]-middle_point[0], points[:,1]-middle_point[1])))[0]
    return np.abs(np.pi/2 - angle_diff) < angle_tol

In [ ]:
skeleton = skeletonize(thersholded)
edges = measure.find_contours(skeleton, 0.5)

# Initialize the graph
G = nx.Graph()

# Add the nodes to the graph
for i in range(skeleton.shape[0]):
    for j in range(skeleton.shape[1]):
        if skeleton[i, j]:
            G.add_node((i, j))

# Add the edges to the graph
for edge in edges:
    edge = np.round(edge).astype(int)
    for i in range(len(edge)-1):
        u = (edge[i][0], edge[i][1])
        v = (edge[i+1][0], edge[i+1][1])
        G.add_edge(u, v)

# Plot the graph
pos = {node: node[::-1] for node in G.nodes()}
nx.draw(G, pos=pos, with_labels=False, node_size=1, edge_color='r', width=0.5)

In [ ]:
points = [(edge[0],edge[-1]) for edge in edges]
points

In [ ]:
from scipy.spatial import Delaunay
points_del = np.array((points[1],points[0])).transpose()
tri = Delaunay(points_del)

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(facecolor='black')
ax.set_facecolor('black')
ax.triplot(points_del[:,0], points_del[:,1], tri.simplices,color="white")

ax.plot(points_del[:,0], points_del[:,1], 'o',color="red")
plt.savefig("delaunay.png")
